In [1]:
# Import libraries
import awswrangler as wr
import numpy as np
import pandas as pd
#%%
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np
from sklearn.metrics import mean_absolute_error
print(tf.__version__)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

2021-12-09 19:03:27.129625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-09 19:03:27.129653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.6.0


In [2]:
# Set paths to file locations on AWS S3 bucket
meth_path = 's3://adisiddharth/M_batch1_batch2_select_no_ffpe_corrected.csv'
pheno_path = 's3://adisiddharth/phenotype_mappings.csv'

In [3]:
# read the files 
meth = wr.s3.read_csv(meth_path) # methylation data
pheno = wr.s3.read_csv(pheno_path) # Endo case v control and methylation data

In [4]:
meth.head()

,Unnamed: 0,201904640089_R01C01,201904640089_R02C01,201904640089_R03C01,201904640089_R04C01,201904640089_R05C01,201904640089_R06C01,201904640089_R08C01,201904640093_R01C01,201904640093_R02C01,...,202878330112_R05C01,202878330112_R06C01,202878330112_R08C01,202878330113_R01C01,202878330113_R02C01,202878330113_R04C01,202878330113_R05C01,202878330113_R06C01,202878330113_R07C01,202878330113_R08C01
0,cg21870274,2.039036,1.960953,2.091686,2.681174,1.765793,2.209447,1.421427,2.606704,1.929786,...,2.070839,2.426515,1.833213,1.610270,2.241103,2.002037,2.526619,2.039211,2.454886,1.321602
1,cg09499020,-0.721799,-0.483177,-0.779545,-0.641505,-1.132911,-0.822985,-0.657523,-0.700859,-0.997561,...,-0.891789,-1.034995,-0.587403,-0.715588,-0.409556,-0.471881,-0.873174,-0.405199,-0.621589,-0.731240
2,cg16535257,-0.411768,-0.821167,-1.158785,-0.856451,-0.136729,-1.236592,-0.713542,-0.948928,-0.599383,...,-0.881235,-0.892351,-1.097712,-0.479917,-0.462617,-0.390668,-0.595121,-0.660783,-0.821571,-0.836415
3,cg06325811,-1.484742,-1.421923,-1.266587,-1.487919,-1.686892,-1.329567,-1.649779,-1.470192,-1.564335,...,-1.589751,-1.406485,-1.751701,-1.517674,-1.665049,-1.663901,-1.617496,-1.210595,-1.509218,-1.426176
4,cg08258224,3.083376,3.035740,2.730783,2.934617,3.200332,2.627382,3.127768,3.116640,2.712387,...,2.579359,2.956215,2.549118,2.925220,2.954624,3.036522,2.937856,3.064265,2.916033,3.559061


In [5]:
# rename columns 
pheno = pheno.rename(columns={col: col.replace('.', '_') for col in pheno.columns})
pheno = pheno.rename(columns={'Unnamed: 0': 'meth_id'})
vars_of_interest = ['meth_id', 'Endometriosis__Yes_No_', 'Cycle_phase_for_Analysis', 'Endometriosis_stage_grouped__I_II____III_IV_', 'Institute_for_Analysis', 'Batch']
pheno = pheno.loc[:, vars_of_interest]
pheno = pheno.rename(columns={'Endometriosis__Yes_No_': 'Endometriosis', 'Endometriosis_stage_grouped__I_II____III_IV_': 'Stage', 'Institute_for_Analysis': 'Institute', 'Cycle_phase_for_Analysis': 'Cycle_Phase'})

In [6]:
pheno.head()

,meth_id,Endometriosis,Cycle_Phase,Stage,Institute,Batch
0,201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1
1,201904640089_R02C01,No,PE,CONTROL,UCSF,1
2,201904640089_R03C01,Yes,ESE,III-IV,UCSF,1
3,201904640089_R04C01,Yes,MSE,I-II,"IMB, UQ",1
4,201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1


In [7]:
meth = meth.rename(columns={'Unnamed: 0': 'probe'})
meth = meth.set_index('probe')
meth = meth.transpose()

In [12]:
# merge both dataframes on methylation ID 
merged_df = pheno.merge(meth, left_on='meth_id', right_on=meth.index, how='inner')
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [13]:
# Change Endo Yes/No to binary
merged_df['Endometriosis_int'] = np.where(merged_df['Endometriosis'] == 'Yes', 1, merged_df['Endometriosis'])
merged_df['Endometriosis_int'] = np.where(merged_df['Endometriosis_int'] == 'No', 0, merged_df['Endometriosis_int'])

In [16]:
# Filter features for inclusion
merged_df = merged_df[merged_df['Cycle_Phase'].isin(['PE', 'ESE', 'MSE', 'LSE', 'Menstrual'])]
merged_df = merged_df[merged_df['Endometriosis'].isin(['Yes', 'No'])]
merged_df = merged_df[merged_df['Stage'].isin(['III-IV', 'CONTROL', 'I-II'])]


# Aggregated ESE, MSE, LSE cycle phases together
merged_df.loc[merged_df['Cycle_Phase'].isin(['ESE', 'MSE', 'LSE']), 'Cycle_Phase'] = 'SE'

In [17]:
# set index as methylation ID
merged_df = merged_df.set_index(merged_df['meth_id'])

In [19]:
# Split dataset into features and labels
x = merged_df.iloc[:, 6:593773] # CPG probes
y = merged_df.iloc[:, 593773] # Endo Case vs Control 

In [20]:
# Split data into train and test (80/20) stratified by cycle phase 
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=merged_df['Cycle_Phase'], test_size=0.2)

In [24]:
# Change labels to int for linear regression 
y_train = y_train.astype('int64')

# Transpose features df for lin regress
X_train = X_train.T

In [25]:
# set up function for feature extraction using linear regression
import scipy 
from scipy.stats import linregress
def calc_slope(row):
    a = scipy.stats.linregress(y_train, row)
    return pd.Series(a._asdict())

In [26]:
# Perform feature extraction
feat_extract = x.join(X_train.apply(calc_slope,axis=1))
feat_extract

,201904640089_R01C01,201904640089_R02C01,201904640089_R03C01,201904640089_R04C01,201904640089_R05C01,201904640089_R06C01,201904640089_R08C01,201904640093_R01C01,201904640093_R02C01,201904640093_R03C01,...,202184900076_R03C01,202184900076_R04C01,202184900076_R06C01,202184900059_R04C01,slope,intercept,rvalue,pvalue,stderr,intercept_stderr
cg21870274,2.039036,1.960953,2.091686,2.681174,1.765793,2.209447,1.421427,2.606704,1.929786,2.000012,...,2.137374,2.927024,2.334874,2.867903,-0.011381,2.036525,-0.015637,0.656564,0.025586,0.020684
cg09499020,-0.721799,-0.483177,-0.779545,-0.641505,-1.132911,-0.822985,-0.657523,-0.700859,-0.997561,0.174104,...,-0.750536,-0.833248,-0.675145,-1.004082,-0.000104,-0.750984,-0.000200,0.995465,0.018225,0.014733
cg16535257,-0.411768,-0.821167,-1.158785,-0.856451,-0.136729,-1.236592,-0.713542,-0.948928,-0.599383,0.047817,...,-1.227665,-1.561893,-0.769275,-0.192639,-0.056789,-0.587014,-0.071423,0.042006,0.027883,0.022541
cg06325811,-1.484742,-1.421923,-1.266587,-1.487919,-1.686892,-1.329567,-1.649779,-1.470192,-1.564335,-1.625282,...,-1.820904,-0.689921,-2.226923,-1.128766,0.037226,-1.489571,0.069024,0.049417,0.018916,0.015292
cg08258224,3.083376,3.035740,2.730783,2.934617,3.200332,2.627382,3.127768,3.116640,2.712387,2.516342,...,2.487833,2.775226,3.475557,2.545495,-0.013915,2.867286,-0.022730,0.518033,0.021519,0.017396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg16667333,1.159705,1.182570,1.387760,1.122688,1.502455,1.330915,0.945207,1.456037,0.971213,2.033534,...,1.329556,1.256327,0.597387,1.592872,-0.001158,1.111537,-0.002005,0.954535,0.020301,0.016412
cg07211220,3.365449,3.925920,4.148340,3.709386,4.124252,3.873856,4.216427,4.116416,3.936559,4.203389,...,3.873212,4.096688,3.972310,4.017213,-0.009376,3.954546,-0.017861,0.611531,0.018454,0.014918
cg01566232,-0.171601,-0.255960,0.057897,-0.145001,-0.097553,0.234588,-0.225311,-0.090310,-0.115908,0.919079,...,0.070319,-0.334140,-0.074274,-0.042516,-0.021006,-0.152159,-0.057088,0.104254,0.012916,0.010441
cg15894539,2.604686,3.184186,3.171590,2.930336,3.265681,2.823836,3.487068,3.176193,3.267671,3.296884,...,3.389347,3.277219,3.015352,3.107948,-0.020287,3.188101,-0.043195,0.219156,0.016497,0.013336


In [27]:
#Benjamini-Hochberg filtering p-values
feat_extract = feat_extract.sort_values(by='pvalue')
feat_extract['rank'] = feat_extract.reset_index().index + 1
feat_extract['(I/m)Q'] = (feat_extract['rank']/len(feat_extract))*0.05
feat_extract['(I/m)Q - p'] = feat_extract['(I/m)Q'] - feat_extract['pvalue']
feat_extract['nominally_sig'] = np.where(feat_extract['pvalue'] < 0.05, 1, 0)
feat_extract['BH_sig'] = np.where(feat_extract['(I/m)Q - p'] < 0, 0, 1)

In [28]:
feat_extract['nominally_sig'].sum(), feat_extract['BH_sig'].sum()

(47014, 8)

In [29]:
# Filter for nominally significant probes
feat_extract_NomSig = feat_extract[feat_extract['nominally_sig'] == 1]

In [30]:
# Get list of probes signficantly correlated to labels in training set
feat_probes = feat_extract_NomSig.index.tolist()
feat_probes

['cg02069524',
 'cg00191665',
 'cg11136960',
 'cg05587394',
 'cg26561401',
 'cg00850551',
 'cg03945895',
 'cg09228183',
 'cg00620024',
 'cg00168191',
 'cg05504729',
 'cg24371954',
 'cg10598496',
 'cg26548834',
 'cg03466339',
 'cg19043188',
 'cg24698222',
 'cg19731777',
 'cg24183793',
 'cg09008179',
 'cg27180755',
 'cg02011723',
 'cg24209045',
 'cg07693365',
 'cg00010187',
 'cg09935160',
 'cg01341572',
 'cg06698243',
 'cg02322492',
 'cg02596779',
 'cg26658396',
 'cg07842327',
 'cg04132252',
 'cg06096184',
 'cg15525302',
 'cg15119597',
 'cg00804517',
 'cg09512823',
 'cg06045576',
 'cg13641341',
 'cg02177811',
 'cg20308684',
 'cg03298789',
 'cg22822824',
 'cg19165390',
 'cg11255469',
 'cg03189453',
 'cg25856265',
 'cg18418457',
 'cg06700110',
 'cg04261496',
 'cg23213894',
 'cg19861623',
 'cg01318506',
 'cg20116128',
 'cg07979675',
 'cg11228607',
 'cg13910263',
 'cg02721176',
 'cg17071409',
 'cg10066898',
 'cg21549530',
 'cg10463299',
 'cg22815118',
 'cg11151402',
 'cg22461472',
 'cg071031

In [31]:
# export CPG probes list as txt 
with open("Endo_feature_extract_CPG.txt", "w") as output:
    output.write(str(feat_probes))

In [32]:
# Export feature extracte df as csv
feat_extract_NomSig.to_csv('Endometriosis_feature_extract.csv')